In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from collections import Counter

In [2]:
with open('inputs/19.txt', 'r') as file:
    inp = file.read()[:-1]

In [3]:
def all_rot(vec):
    vecs = []
    for xy_rot in np.radians([[0,0,0],[90,0,0],[180,0,0],[270,0,0],[0,90,0],[0,-90,0]]):
        rotation = R.from_rotvec(xy_rot)
        rotated_vec = rotation.apply(vec)
        for z_rot in np.radians([[0,0,0],[0,0,90],[0,0,180],[0,0,270]]):
            rotation = R.from_rotvec(z_rot)
            rotated_vec = rotation.apply(rotated_vec)
            vecs.append(rotated_vec)
    return np.array(vecs)

In [4]:
# 1 and 2
coo = [[[int(k) for k in j.split(",")] for j in i.split("\n") if "," in j] for i in inp.split("\n\n")]

scanners = []
beacons = set()
pos = np.array([0,0,0])
rel = np.array(coo[0])
done = [0]
queue = [[pos, rel]]

while queue:
    (pos, rel) = queue.pop()
    scanners.append(pos)
    for i in rel: beacons.add(tuple(i+np.array(pos)))
    for scan_nr,i in enumerate(coo):
        rotations = np.round(np.array(all_rot(i)))
        if scan_nr not in done:
            for r in rotations:
                distances = (rel[:,None,:]-r[None,:,:]).reshape(-1,3)
                only_coords = zip(distances[:,0], distances[:,1], distances[:,2])
                coor, counts = Counter(only_coords).most_common()[0]
                if counts >= 12:
                    done += [scan_nr]
                    queue.append([pos+np.round(np.array(coor)).astype(int), r*1])
                    break
print(len(beacons))
print(np.max(np.sum(np.abs(np.array(scanners)[:,None,:]-np.array(scanners)[None,:,:]), axis=2)))

338
9862
